In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
import plotly.express as px
import os

In [3]:
CSV_PATH = "/Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/JH_Head_CT/method_vs_gold_per_scan.csv"
df = pd.read_csv(CSV_PATH)
df['icv_ratio'] = df['icv_ml_pred'] / df['icv_ml_ref']
display(df.head())

,patient_id,stem,method_A,method_B,dice,iou,sensitivity,specificity,msd_mm,hd95_mm,icv_ml_ref,icv_ml_pred,delta_icv_ml,gold_path,pred_path,above_thresh,within_5ml,icv_ratio
0,1,01_BRAIN_1_Anonymized,Brainchop,GoldManual,0.968,0.939,0.992,0.991,0.940,2.619,1225.127,1284.199,59.072,/dcs05/ciprian/smart/mistie_3/jhu_manual_segme...,/dcs05/ciprian/smart/mistie_3/jhu_manual_segme...,0,0,1.048217
1,1,01_BRAIN_1_Anonymized,HD-CTBET,GoldManual,0.949,0.902,0.991,0.984,2.014,4.469,1225.127,1335.502,110.375,/dcs05/ciprian/smart/mistie_3/jhu_manual_segme...,/dcs05/ciprian/smart/mistie_3/jhu_manual_segme...,0,0,1.090093
2,1,01_BRAIN_1_Anonymized,SynthStrip,GoldManual,0.951,0.906,0.998,0.983,1.770,3.013,1225.127,1346.028,120.901,/dcs05/ciprian/smart/mistie_3/jhu_manual_segme...,/dcs05/ciprian/smart/mistie_3/jhu_manual_segme...,0,0,1.098684
3,1,01_BRAIN_1_Anonymized,CTBET,GoldManual,0.992,0.984,0.990,0.999,0.206,1.797,1225.127,1220.469,-4.658,/dcs05/ciprian/smart/mistie_3/jhu_manual_segme...,/dcs05/ciprian/smart/mistie_3/jhu_manual_segme...,1,1,0.996198
4,1,01_BRAIN_1_Anonymized,Robust-CTBET,GoldManual,0.995,0.990,0.990,1.000,0.120,0.898,1225.127,1213.579,-11.548,/dcs05/ciprian/smart/mistie_3/jhu_manual_segme...,/dcs05/ciprian/smart/mistie_3/jhu_manual_segme...,1,0,0.990574


In [ ]:
THRESHOLDS = [0.95, 0.97, 0.99]
outdir = Path("/Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/derived_metrics")
outdir.mkdir(parents=True, exist_ok=True)

# For gold-standard comparisons: method_A is the algorithm, method_B is the gold reference.
# We don't canonicalize pairs here because order matters (A=method, B=gold).
data_root = Path("/Users/rushil/brain_extraction/results/quantitative/gold_std_comparison")
csv_paths = sorted(data_root.rglob('method_vs_gold_per_scan*.csv'))
if not csv_paths:
    print(f'No input CSVs found under {data_root}')

for csv_path in csv_paths:
    dataset = csv_path.parent.name
    print(f'Processing dataset: {dataset} -> {csv_path}')
    df_ds = pd.read_csv(csv_path)
    # compute icv_ratio if columns present
    if 'icv_ml_pred' in df_ds.columns and 'icv_ml_ref' in df_ds.columns:
        df_ds['icv_ratio'] = df_ds['icv_ml_pred'] / df_ds['icv_ml_ref']

    methods = sorted(df_ds['method_A'].unique())
    rows_all = []
    # For each method (method_A compared to gold in method_B), count rows and threshold passes
    for m in methods:
        method_df = df_ds[df_ds['method_A'] == m].copy()
        n_comparisons = int(len(method_df))
        for t in THRESHOLDS:
            n_ge = int((method_df['dice'] >= t).sum())
            pct = (n_ge / n_comparisons * 100.0) if n_comparisons else np.nan
            rows_all.append({'method': m, 'threshold': float(t), 'n_comparisons': n_comparisons, 'n_ge': n_ge, 'pct_ge': float(pct)})
            method_dir = outdir / dataset
            method_dir.mkdir(parents=True, exist_ok=True)
    # Save combined CSV and pivot matrix per dataset
    combined_df = pd.DataFrame(rows_all)
    pivot = combined_df.pivot(index='method', columns='threshold', values='pct_ge').reindex(index=methods)
    pivot.to_csv(outdir / dataset / 'dice_matrix_by_method_thresholds.csv')
    display(Markdown(f'### Results for {dataset}'))
    display(pivot)


Processing dataset: CERMEP-IDB-MRXFDG -> /Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/CERMEP-IDB-MRXFDG/method_vs_gold_per_scan_synth.csv


### Results for CERMEP-IDB-MRXFDG

threshold,0.95,0.97,0.99
method,,,
Brainchop,85.0,30.0,0.0
CTBET,0.0,0.0,0.0
CT_BET,100.0,95.0,5.0
CTbet_Docker,85.0,55.0,0.0
HD-CTBET,90.0,0.0,0.0
Robust-CTBET,85.0,80.0,80.0
SynthStrip,50.0,0.0,0.0


Processing dataset: JH_Head_CT -> /Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/JH_Head_CT/method_vs_gold_per_scan.csv


### Results for JH_Head_CT

threshold,0.95,0.97,0.99
method,,,
Brainchop,86.842105,21.052632,0.000000
CTBET,100.000000,100.000000,86.842105
CT_BET,100.000000,100.000000,14.285714
CTbet_Docker,100.000000,100.000000,52.631579
HD-CTBET,65.789474,0.000000,0.000000
Robust-CTBET,100.000000,100.000000,97.368421
SynthStrip,44.736842,0.000000,0.000000


Processing dataset: MISTIE_III -> /Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/MISTIE_III/method_vs_gold_per_scan_mistie3.csv


### Results for MISTIE_III

threshold,0.95,0.97,0.99
method,,,
Brainchop,20.000000,10.000000,0.0
CTBET,60.000000,50.000000,0.0
CT_BET,71.428571,14.285714,0.0
CTbet_Docker,100.000000,80.000000,0.0
HD-CTBET,30.000000,0.000000,0.0
Robust-CTBET,80.000000,70.000000,0.0
SynthStrip,20.000000,0.000000,0.0


In [18]:
THRESHOLDS = [2.5, 5.0, 10.0]
outdir = Path("/Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/derived_metrics")
outdir.mkdir(parents=True, exist_ok=True)

data_root = Path("/Users/rushil/brain_extraction/results/quantitative/gold_std_comparison")
csv_paths = sorted(data_root.rglob('method_vs_gold_per_scan*.csv'))
if not csv_paths:
    print(f'No input CSVs found under {data_root}')

for csv_path in csv_paths:
    dataset = csv_path.parent.name
    print(f'Processing dataset: {dataset} -> {csv_path}')
    df_ds = pd.read_csv(csv_path)

    methods = sorted(df_ds['method_A'].unique())
    rows_all = []
    # For each method (method_A compared to gold in method_B), count rows and threshold passes
    for m in methods:
        method_df = df_ds[df_ds['method_A'] == m].copy()
        n_comparisons = int(len(method_df))
        for t in THRESHOLDS:
            n_ge = int((method_df['delta_icv_ml'].abs() <= t).sum())
            pct = (n_ge / n_comparisons * 100.0) if n_comparisons else np.nan
            rows_all.append({'method': m, 'threshold': float(t), 'n_comparisons': n_comparisons, 'n_ge': n_ge, 'pct_ge': float(pct)})
            method_dir = outdir / dataset
            method_dir.mkdir(parents=True, exist_ok=True)
    # Save combined CSV and pivot matrix per dataset
    combined_df = pd.DataFrame(rows_all)
    pivot = combined_df.pivot(index='method', columns='threshold', values='pct_ge').reindex(index=methods)
    pivot.to_csv(outdir / dataset / 'icv_matrix_by_method_thresholds.csv')
    display(Markdown(f'### Results for {dataset}'))
    display(pivot)


Processing dataset: CERMEP-IDB-MRXFDG -> /Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/CERMEP-IDB-MRXFDG/method_vs_gold_per_scan_synth.csv


### Results for CERMEP-IDB-MRXFDG

threshold,2.5,5.0,10.0
method,,,
Brainchop,0.0,0.0,5.0
CTBET,5.0,5.0,5.0
CT_BET,5.0,15.0,45.0
CTbet_Docker,0.0,0.0,0.0
HD-CTBET,0.0,0.0,0.0
Robust-CTBET,0.0,5.0,70.0
SynthStrip,0.0,0.0,0.0


Processing dataset: JH_Head_CT -> /Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/JH_Head_CT/method_vs_gold_per_scan.csv


### Results for JH_Head_CT

threshold,2.5,5.0,10.0
method,,,
Brainchop,2.631579,7.894737,13.157895
CTBET,28.947368,57.894737,86.842105
CT_BET,7.142857,21.428571,50.000000
CTbet_Docker,15.789474,47.368421,71.052632
HD-CTBET,0.000000,2.631579,5.263158
Robust-CTBET,0.000000,7.894737,50.000000
SynthStrip,2.631579,2.631579,5.263158


Processing dataset: MISTIE_III -> /Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/MISTIE_III/method_vs_gold_per_scan_mistie3.csv


### Results for MISTIE_III

threshold,2.5,5.0,10.0
method,,,
Brainchop,0.0,0.0,10.000000
CTBET,0.0,0.0,0.000000
CT_BET,0.0,0.0,14.285714
CTbet_Docker,10.0,20.0,20.000000
HD-CTBET,0.0,0.0,10.000000
Robust-CTBET,0.0,0.0,10.000000
SynthStrip,0.0,0.0,10.000000


In [19]:
THRESHOLDS = [0.95, 0.97, 0.99]
outdir = Path("/Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/derived_metrics")
outdir.mkdir(parents=True, exist_ok=True)

data_root = Path("/Users/rushil/brain_extraction/results/quantitative/gold_std_comparison")
csv_paths = sorted(data_root.rglob('method_vs_gold_per_scan*.csv'))
if not csv_paths:
    print(f'No input CSVs found under {data_root}')

for csv_path in csv_paths:
    dataset = csv_path.parent.name
    print(f'Processing dataset: {dataset} -> {csv_path}')
    df_ds = pd.read_csv(csv_path)

    # require icv columns to compute ratio
    if 'icv_ml_pred' not in df_ds.columns or 'icv_ml_ref' not in df_ds.columns:
        print(f"Skipping {csv_path} - missing icv_ml_pred/icv_ml_ref")
        continue
    df_ds['icv_ratio'] = df_ds['icv_ml_pred'] / df_ds['icv_ml_ref']

    methods = sorted(df_ds['method_A'].unique())
    combined_rows = []

    # For each method (method_A compared to gold in method_B), count rows within icv_ratio thresholds
    for m in methods:
        method_df = df_ds[df_ds['method_A'] == m].copy()
        n_comparisons = int(len(method_df))
        rows_m = []
        for t in THRESHOLDS:
            # interpret threshold t as closeness to 1.0 (e.g. t=0.95 => within 5% => |ratio-1| <= 0.05)
            tol = 1.0 - float(t)
            n_within = int((method_df['icv_ratio'].sub(1.0).abs() <= tol).sum())
            pct_within = (n_within / n_comparisons * 100.0) if n_comparisons else np.nan
            row = {'method': m, 'threshold': float(t), 'n_comparisons': n_comparisons, 'n_within': n_within, 'pct_within': float(pct_within)}
            rows_m.append(row)
            combined_rows.append(row)

        # save per-method CSV under derived_metrics/<dataset>/<method>/
        safe_method = str(m).replace('/', '_')
        method_dir = outdir / dataset / safe_method
        method_dir.mkdir(parents=True, exist_ok=True)
        pd.DataFrame(rows_m).to_csv(method_dir / 'icv_ratio_within_thresholds_by_method.csv', index=False)

    # Save combined CSV and pivot matrix per dataset
    combined_df = pd.DataFrame(combined_rows)
    combined_df.to_csv(outdir / dataset / 'icv_ratio_within_thresholds_all_methods.csv', index=False)
    pivot = combined_df.pivot(index='method', columns='threshold', values='pct_within').reindex(index=methods)
    pivot.to_csv(outdir / dataset / 'icv_ratio_matrix_by_method_thresholds.csv')
    display(Markdown(f'### ICV ratio results for {dataset}'))
    display(pivot)

Processing dataset: CERMEP-IDB-MRXFDG -> /Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/CERMEP-IDB-MRXFDG/method_vs_gold_per_scan_synth.csv


### ICV ratio results for CERMEP-IDB-MRXFDG

threshold,0.95,0.97,0.99
method,,,
Brainchop,75.0,35.0,5.0
CTBET,5.0,5.0,5.0
CT_BET,100.0,100.0,70.0
CTbet_Docker,65.0,45.0,5.0
HD-CTBET,55.0,20.0,0.0
Robust-CTBET,80.0,80.0,80.0
SynthStrip,5.0,0.0,0.0


Processing dataset: JH_Head_CT -> /Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/JH_Head_CT/method_vs_gold_per_scan.csv


### ICV ratio results for JH_Head_CT

threshold,0.95,0.97,0.99
method,,,
Brainchop,76.315789,42.105263,15.789474
CTBET,100.000000,100.000000,92.105263
CT_BET,100.000000,100.000000,64.285714
CTbet_Docker,100.000000,100.000000,86.842105
HD-CTBET,28.947368,15.789474,5.263158
Robust-CTBET,100.000000,100.000000,68.421053
SynthStrip,7.894737,5.263158,5.263158


Processing dataset: MISTIE_III -> /Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/MISTIE_III/method_vs_gold_per_scan_mistie3.csv


### ICV ratio results for MISTIE_III

threshold,0.95,0.97,0.99
method,,,
Brainchop,60.000000,40.000000,10.000000
CTBET,70.000000,20.000000,0.000000
CT_BET,57.142857,42.857143,14.285714
CTbet_Docker,100.000000,60.000000,20.000000
HD-CTBET,30.000000,10.000000,10.000000
Robust-CTBET,70.000000,50.000000,20.000000
SynthStrip,20.000000,10.000000,10.000000
